# Simple AC Detector

This notebook contains code that will collect data from the microphone and the environmental sensor to try to determine if an air conditioner has been turned on.

### Sound as an Indicator

A baseline volume level is determined during the calibration process. Afterwards, the current volume of the room is compared to this baseline value. If the volume has significantly increased then it will assume the AC has been turned on. This method is very simple be not very accurate. It is easily susceptible to false positives as it attributes any increase in volume to an AC.

### Temperature as an Indicator

A baseline temperature is determined during the calibration process. Afterwards, the current temperature of the room is compared to this baseline value. If the temperature has significantly decreased then it will assume the AC has been turned on. 

The methods used to determine if an AC has been turned on are very simple but not very accurate. To improve upon this methodology, the data from both sensors can be combined to make a better guess. Additionally, the audio data could undergo a more complex analysis process, such as matching the current sound in the room to a known AC sound.

---

## The Code

### Setup

In [16]:
from waggle.data.audio import Microphone
import time
from scipy.io import wavfile
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import bme680

using backwards compatible implementation of time_ns


In [17]:
microphone = Microphone()

sensor = bme680.BME680()
sensor.set_humidity_oversample(bme680.OS_2X)
sensor.set_pressure_oversample(bme680.OS_4X)
sensor.set_temperature_oversample(bme680.OS_8X)
sensor.set_filter(bme680.FILTER_SIZE_3)
sensor.set_gas_status(bme680.DISABLE_GAS_MEAS)
sensor.set_gas_heater_temperature(320)
sensor.set_gas_heater_duration(150)
sensor.select_gas_heater_profile(0)

## Helper Functions

Records an audio sample for the given duration of time and returns the recording's data.

In [18]:
def getAudioSample(duration):
    sample = microphone.record(duration)
    sample.save("ACDetector.wav")
    (samplerate, data) = wavfile.read('ACDetector.wav')
    return data


Collects temperature readings once a second for the given duration and returns the readings in an array.

In [19]:
def getTempSample(duration):
    enviData = []
    
    for i in range(1, duration):
        if sensor.get_sensor_data():
            reading = sensor.data.temperature
            enviData = np.append(enviData, [reading], axis=0)
            
        time.sleep(1)
        
    return enviData

Determines if the AC is on by checking if the given value is within 25% of the difference between the on and off value of the AC.

In [20]:
def calcACState(off, on, current):
    difference = on - off
    deviation = difference * 0.25
    
    if difference > 0:
        if current >= on - deviation:
            return True
        
    elif difference < 0:
        if current <= on + deviation:
            return True
   
    return False
    

## Calibration

The calibration part of this notebook should only be run when the AC known to be off.

### AC Off

In [22]:
print("Calibrating Microphone with AC OFF...")
audioData = getAudioSample(10)
mic_AC_OFF = np.average(np.absolute(audioData))
print("Amplitude with AC off: ", mic_AC_OFF)

print("Calibrating Envi Sensor with AC OFF...")
enviData = getTempSample(10)
temp_AC_OFF = np.average(enviData)
print("Temperature with AC OFF: ", temp_AC_OFF)

Calibrating Microphone with AC OFF...
Amplitude with AC off:  405.85085833333335
Calibrating Envi Sensor with AC OFF...
Temperature with AC OFF:  25.413333333333334


### AC On

In [24]:
print("Calibrating Microphone with AC ON...")
audioData = getAudioSample(10)
mic_AC_ON = np.average(np.absolute(audioData))
print("Amplitude with AC ON: ", mic_AC_ON)

print("Calibrating Envi Sensor with AC ON...")
enviData = getTempSample(10)
temp_AC_ON = np.average(enviData)
print("Temperature with AC ON: ", temp_AC_ON)

Calibrating Microphone with AC ON...
Amplitude with AC ON:  9985.901629166667
Calibrating Envi Sensor with AC ON...
Temperature with AC ON:  25.29222222222222


## Detect an AC

This loop will alternate between using the microphone and the environmental sensor to try to determine if an AC has been turned on in the last 10 seconds.

In [32]:
while True:
    # Recording Audio
    audioData = getAudioSample(10)
    avgAmp = np.average( np.absolute(audioData))
    print("Currnt avg Amplitude: ", avgAmp)
    
    # Check if AC is on based on sound
    if calcACState(mic_AC_OFF, mic_AC_ON, avgAmp):
        print("AC is now on based on mic")
    else:
        print("AC is now off based on mic")
    
     # Collecting Enviornmental Data
    enviData = getTempSample(10)
    avgTemp = np.average(enviData)
    print("Current avg temp:", avgTemp)
    
    # Check if AC is on based on temperature
    if calcACState(temp_AC_OFF, temp_AC_ON, avgTemp):
        print("AC is now on based on envi")
    else:
        print("AC is now off based on envi")

KeyboardInterrupt: 